In [1]:
import requests

In [2]:
standings_url = "https://liquipedia.net/valorant/VCT/2022/North_America/Stage_2/Challengers/Group_Stage"

In [3]:
data = requests.get(standings_url)

In [4]:
from bs4 import BeautifulSoup

In [5]:
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.grouptable')[:2]
teams = []
for group in standings_table:
    links = group.find_all('a')
    links = [l.get("href") for l in links]
    for team in links:
        teams.append(team)
        
teams = sorted(list(set(teams)))
teams

['/valorant/100_Thieves',
 '/valorant/Cloud9',
 '/valorant/Evil_Geniuses',
 '/valorant/FaZe_Clan',
 '/valorant/Ghost_Gaming',
 '/valorant/Luminosity_Gaming',
 '/valorant/NRG',
 '/valorant/OpTic_Gaming',
 '/valorant/Sentinels',
 '/valorant/TSM',
 '/valorant/The_Guard',
 '/valorant/XSET']

In [6]:
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.collapsible')[0]
short = standings_table.find_all('a')
short = [l.text for l in short]
shortteams = []
for team in short:
    shortteams.append(team)
shortteams = list(set(shortteams))
shortteams

['',
 'LG',
 'OPTC',
 'EG',
 'NRG',
 'Sideshow',
 'Haven',
 'Bren',
 'GHST',
 'Icebox',
 'Breeze',
 'IHOLDSHIFT',
 'EsportsDoug',
 'Bind',
 'TSM',
 'Fracture',
 '100T',
 'XSET',
 'TGRD',
 'FaZe',
 'dusT',
 'SEN',
 'Split',
 'C9',
 'Ballatw',
 'Ascent']

In [7]:
shorthand = ['TSM','XSET','LG','SEN','100T','NRG','TGRD','OPTC','EG','C9','Ghost','FaZe']
shorthand = sorted(shorthand)
shorthand

['100T',
 'C9',
 'EG',
 'FaZe',
 'Ghost',
 'LG',
 'NRG',
 'OPTC',
 'SEN',
 'TGRD',
 'TSM',
 'XSET']

In [8]:
i = shorthand.index("TGRD")
j = shorthand.index("TSM")
shorthand[i], shorthand[j] = shorthand[j], shorthand[i]
shorthand

['100T',
 'C9',
 'EG',
 'FaZe',
 'Ghost',
 'LG',
 'NRG',
 'OPTC',
 'SEN',
 'TSM',
 'TGRD',
 'XSET']

rng read everytime?
['/valorant/100_Thieves',
 '/valorant/Cloud9',
 '/valorant/Evil_Geniuses',
 '/valorant/FaZe_Clan',
 '/valorant/Ghost_Gaming',
 '/valorant/Luminosity_Gaming',
 '/valorant/NRG',
 '/valorant/OpTic_Gaming',
 '/valorant/Sentinels',
 '/valorant/TSM',
 '/valorant/The_Guard',
 '/valorant/XSET']

In [9]:
len(shorthand),len(teams)

(12, 12)

In [10]:
team_urls = [f"https://liquipedia.net{l}/Matches" for l in teams]
team_urls

['https://liquipedia.net/valorant/100_Thieves/Matches',
 'https://liquipedia.net/valorant/Cloud9/Matches',
 'https://liquipedia.net/valorant/Evil_Geniuses/Matches',
 'https://liquipedia.net/valorant/FaZe_Clan/Matches',
 'https://liquipedia.net/valorant/Ghost_Gaming/Matches',
 'https://liquipedia.net/valorant/Luminosity_Gaming/Matches',
 'https://liquipedia.net/valorant/NRG/Matches',
 'https://liquipedia.net/valorant/OpTic_Gaming/Matches',
 'https://liquipedia.net/valorant/Sentinels/Matches',
 'https://liquipedia.net/valorant/TSM/Matches',
 'https://liquipedia.net/valorant/The_Guard/Matches',
 'https://liquipedia.net/valorant/XSET/Matches']

In [11]:
data = requests.get(team_urls[0])

In [12]:
team_name = team_urls[0].split("/")[-2]
team_name

'100_Thieves'

In [13]:
import pandas as pd
matches = pd.read_html(data.text)[0]
matches = matches.drop(columns=['Tournament', 'VOD'])
matches = matches.rename(columns={"Tournament.1": "Tournament", "vs. Team": "Opponent"})
matches.insert(6, "Team", shorthand[0])
matches.head()

,Date,Time,Tier,Type,Tournament,Score,Team,Opponent
0,2022-06-05,20:00 UTC,A-Tier,Online,VCT 2022: North America Challengers 2 - Group ...,2 : 0,100T,NRG
1,2022-05-28,18:15 EDT,A-Tier,Online,VCT 2022: North America Challengers 2 - Group ...,2 : 0,100T,XSET
2,2022-05-21,16:15 EDT,A-Tier,Online,VCT 2022: North America Challengers 2 - Group ...,0 : 2,100T,TSM
3,2022-05-13,16:20 EDT,A-Tier,Online,VCT 2022: North America Challengers 2 - Group ...,2 : 0,100T,TGRD
4,2022-05-07,17:45 EDT,A-Tier (Qual.),Online,VCT 2022: NA Stage 2 Challengers - Open Qualif...,2 : 0,100T,AKRW


In [14]:
all_matches = []

In [15]:
for team_url in team_urls:
    team_name = team_url.split("/")[-2].replace("_", " ")
    data = requests.get(team_url)
    matches = pd.read_html(data.text)[0]
    matches = matches.drop(columns=['Tournament', 'VOD'])
    matches = matches.rename(columns={"Tournament.1": "Tournament", "vs. Team": "Opponent"})
    matches["Team"] = team_name
    all_matches.append(matches)

In [16]:
len(all_matches)

12

In [17]:
match_df = pd.concat(all_matches)

In [18]:
match_df.columns = [c.lower() for c in match_df.columns]

In [19]:
match_df.head()

,date,time,tier,type,tournament,score,opponent,team
0,2022-06-05,20:00 UTC,A-Tier,Online,VCT 2022: North America Challengers 2 - Group ...,2 : 0,NRG,100 Thieves
1,2022-05-28,18:15 EDT,A-Tier,Online,VCT 2022: North America Challengers 2 - Group ...,2 : 0,XSET,100 Thieves
2,2022-05-21,16:15 EDT,A-Tier,Online,VCT 2022: North America Challengers 2 - Group ...,0 : 2,TSM,100 Thieves
3,2022-05-13,16:20 EDT,A-Tier,Online,VCT 2022: North America Challengers 2 - Group ...,2 : 0,TGRD,100 Thieves
4,2022-05-07,17:45 EDT,A-Tier (Qual.),Online,VCT 2022: NA Stage 2 Challengers - Open Qualif...,2 : 0,AKRW,100 Thieves


In [20]:
match_df.tail()

,date,time,tier,type,tournament,score,opponent,team
84,2020-11-11,16:00 PST,A-Tier (Qual.),Online,UMG Tournament - Qualifier #1,0 : 2,FS,XSET
85,2020-11-04,20:15 EST,A-Tier,Online,First Strike North America - NSG Tournament,1 : 2,LG,XSET
86,2020-11-04,15:00 EST,A-Tier,Online,First Strike North America - NSG Tournament,0 : 2,C9 B,XSET
87,2020-10-27,17:00 PDT,A-Tier (Qual.),Online,NSG Tournament - Qualifier,0 : 2,Gen.G,XSET
88,2020-10-27,14:00 PDT,A-Tier (Qual.),Online,NSG Tournament - Qualifier,2 : 1,IMT,XSET


In [21]:
match_df['score'] = match_df['score'].str.replace('-',' : ')
new = match_df["score"].str.split(":", expand = True)
match_df["GF"] = new[0]
match_df["GA"] = new[1]
match_df.head()

,date,time,tier,type,tournament,score,opponent,team,GF,GA
0,2022-06-05,20:00 UTC,A-Tier,Online,VCT 2022: North America Challengers 2 - Group ...,2 : 0,NRG,100 Thieves,2,0
1,2022-05-28,18:15 EDT,A-Tier,Online,VCT 2022: North America Challengers 2 - Group ...,2 : 0,XSET,100 Thieves,2,0
2,2022-05-21,16:15 EDT,A-Tier,Online,VCT 2022: North America Challengers 2 - Group ...,0 : 2,TSM,100 Thieves,0,2
3,2022-05-13,16:20 EDT,A-Tier,Online,VCT 2022: North America Challengers 2 - Group ...,2 : 0,TGRD,100 Thieves,2,0
4,2022-05-07,17:45 EDT,A-Tier (Qual.),Online,VCT 2022: NA Stage 2 Challengers - Open Qualif...,2 : 0,AKRW,100 Thieves,2,0


In [22]:
match_df["team"] = match_df["team"].astype(str)

In [23]:
match_df["opponent"] = match_df["opponent"].astype(str)

In [24]:
match_df.dtypes

date          object
time          object
tier          object
type          object
tournament    object
score         object
opponent      object
team          object
GF            object
GA            object
dtype: object

In [25]:
match_df.to_csv("valNA2022stage2matches.csv")